In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
aapl = yf.download(tickers='AAPL', start='2010-01-01')[['Close']]
aapl

[*********************100%%**********************]  1 of 1 completed


,Close
Date,
2010-01-04,7.643214
2010-01-05,7.656429
2010-01-06,7.534643
2010-01-07,7.520714
2010-01-08,7.570714
...,...
2023-10-09,178.990005
2023-10-10,178.389999
2023-10-11,179.800003


In [3]:
def rsi_14(data):
    data = data.copy() 
    data['Change'] = data['Close'].diff()

    data['Gain'] = data['Change'].where(data['Change'] > 0, 0)
    data['Loss'] = -data['Change'].where(data['Change'] < 0, 0)
    
    data['Average Gain'] = np.where(data.index < data.index[14], data['Gain'].rolling(window=14).mean(), np.nan)
    data['Average Loss'] = np.where(data.index < data.index[14], data['Loss'].rolling(window=14).mean(), np.nan)

    for i in range(14, len(data)):
        data['Average Gain'].iloc[i] = (data['Average Gain'].iloc[i-1] * 13 + data['Gain'].iloc[i]) / 14
        data['Average Loss'].iloc[i] = (data['Average Loss'].iloc[i-1] * 13 + data['Loss'].iloc[i]) / 14

    data['RS'] = data['Average Gain'] / data['Average Loss']

    data['RSI'] = (100 - (100/(1+data['RS'])))
    
    return data[['Close', 'RSI']]


In [21]:
def ema(data, window):
    data = data.copy()
    
    first_valid_index = data['Close'].first_valid_index()
    if first_valid_index is not None:
        initial_sma_position = data.index.get_loc(first_valid_index) + window - 1
        if initial_sma_position < len(data):
            initial_sma = data['Close'].rolling(window=window).mean().iloc[initial_sma_position]
        else:
            initial_sma = np.nan
    else:
        initial_sma = np.nan

    multiplier = (2 / (window + 1))
    
    data[f'{window}ema'] = np.nan
    
    if not np.isnan(initial_sma):
        data[f'{window}ema'].iloc[initial_sma_position] = initial_sma
    
        for i in range(initial_sma_position + 1, len(data)):
            data[f'{window}ema'].iloc[i] = (data['Close'].iloc[i] - data[f'{window}ema'].iloc[i-1]) * multiplier + data[f'{window}ema'].iloc[i-1]
    
    return data[[f'{window}ema']]


In [25]:
def macd_strategy(data):
    data = data.copy()

    fastMA = ema(data, 12)
    slowMA = ema(data, 26)
    data['macd'] = fastMA['12ema'] - slowMA['26ema']

    signal_data = data[['macd']].rename(columns={'macd': 'Close'})
    signal_line = ema(signal_data, 9)

    data['signal_line'] = signal_line['9ema']
    
    return data[['macd','signal_line']]


In [26]:
macd_strategy(aapl)

,macd,signal_line
Date,,
2010-01-04,NaN,NaN
2010-01-05,NaN,NaN
2010-01-06,NaN,NaN
2010-01-07,NaN,NaN
2010-01-08,NaN,NaN
...,...,...
2023-10-09,-1.165012,-1.977853
2023-10-10,-0.821506,-1.746583
2023-10-11,-0.430537,-1.483374
